In [1]:
from dotenv import load_dotenv
import requests
import os


# 加载.env文件
load_dotenv()

print(os.environ["SCRAPEOPS_API_KEY"])  # 输出: my_value

9f18efd5-abff-4807-abb5-5f171f726553


In [2]:
import json
import pandas as pd

# Load the processed EVP British English list

with open('evp_british_english_list_processed.json', 'r', encoding='utf-8') as file:
  evp_data = json.load(file)
  
#   # Convert to DataFrame
#   df = pd.DataFrame(evp_data)
  
#   # Display basic information about the DataFrame
#   print(f"Total words: {len(df)}")
#   print("\nColumns in the DataFrame:")
#   print(df.columns.tolist())
#   print("\nFirst 5 rows:")
#   print(df.head())
  
# except FileNotFoundError:
#   print("File 'evp_british_english_list_processed.json' not found.")
# except json.JSONDecodeError:
#   print("Error decoding JSON from the file.")

In [ ]:
len(evp_data)
# Check if all items in evp_data have exactly 1 definition
all_have_one_definition = all(len(item.get('definitions', [])) == 1 for item in evp_data)

# Count items with different numbers of definitions
definition_counts = {}
for item in evp_data:
  num_definitions = len(item.get('definitions', []))
  if num_definitions in definition_counts:
    definition_counts[num_definitions] += 1
  else:
    definition_counts[num_definitions] = 1

print(f"All items have exactly 1 definition: {all_have_one_definition}")
print(f"Distribution of definition counts: {definition_counts}")

# Check a few examples
if len(evp_data) > 0:
  print("\nSample items with their definition counts:")
  for i in range(min(5, len(evp_data))):
    word = evp_data[i].get('word', 'N/A')
    num_defs = len(evp_data[i].get('definitions', []))
    print(f"{word}: {num_defs} definition(s)")
    # Count items with and without definitions
    have_definitions = [item for item in evp_data if len(item.get('definitions', [])) > 0]
    have_no_definitions = [item for item in evp_data if len(item.get('definitions', [])) == 0]

    # print(f"Items with definitions: {len(have_definitions)}")
    # print(f"Items without definitions: {len(have_no_definitions)}")

    # # Save items with and without definitions to separate JSON files
    # with open('evp_have_definitions.json', 'w', encoding='utf-8') as file:
    #   json.dump(have_definitions, file, ensure_ascii=False, indent=2)

    # with open('evp_have_no_definitions.json', 'w', encoding='utf-8') as file:
    #   json.dump(have_no_definitions, file, ensure_ascii=False, indent=2)

    # print(f"Saved {len(have_definitions)} items to 'evp_have_definitions.json'")
    # print(f"Saved {len(have_no_definitions)} items to 'evp_have_no_definitions.json'")


In [ ]:
# Create a counter for different part of speech values
pos_counter = {}
items_by_pos = {}

# Iterate through all items in have_definitions list
for item in have_definitions:
  pos = item.get('part_of_speech', '')
  
  # If this part of speech is already in our counter, increment it
  if pos in pos_counter:
    pos_counter[pos] += 1
    items_by_pos.setdefault(pos, []).append(item)
  # Otherwise, add it to the counter with value 1
  else:
    pos_counter[pos] = 1
    items_by_pos[pos] = [item]

# Sort the results by count in descending order
sorted_pos = sorted(pos_counter.items(), key=lambda x: x[1], reverse=True)





In [20]:
# Display the results
print(f"Total unique part of speech values: {len(pos_counter)}")
print("\nPart of speech distribution:")
for pos, count in sorted_pos:
  pos_display = pos if pos else '(empty string)'
  print(f"{pos_display}: {count} items ({count/len(have_definitions)*100:.2f}%)")

Total unique part of speech values: 13

Part of speech distribution:
noun: 5167 items (33.02%)
phrase: 3611 items (23.08%)
adjective: 2420 items (15.47%)
verb: 2318 items (14.81%)
adverb: 797 items (5.09%)
phrasal verb: 728 items (4.65%)
preposition: 233 items (1.49%)
(empty string): 111 items (0.71%)
pronoun: 101 items (0.65%)
determiner: 85 items (0.54%)
modal verb: 37 items (0.24%)
exclamation: 33 items (0.21%)
auxiliary verb: 6 items (0.04%)


In [ ]:
# import json
# import os

# # Create a directory to store the files
# if not os.path.exists('pos_data'):
#   os.makedirs('pos_data')

# # Save each part of speech data to separate files
# for pos, items in items_by_pos.items():
#   # Use a safe filename (replace empty string with 'empty')
#   filename = pos if pos else 'empty'
  
#   # Save to JSON file
#   file_path = os.path.join('pos_data', f'{filename}.json')
#   with open(file_path, 'w', encoding='utf-8') as file:
#     json.dump(items, file, ensure_ascii=False, indent=2)
  
#   print(f"Saved {len(items)} items with part of speech '{pos or '(empty)'}' to {file_path}")

# # Print a summary
# print(f"\nTotal files created: {len(items_by_pos)}")

Saved 111 items with part of speech '(empty)' to pos_data\empty.json
Saved 2318 items with part of speech 'verb' to pos_data\verb.json
Saved 5167 items with part of speech 'noun' to pos_data\noun.json
Saved 2420 items with part of speech 'adjective' to pos_data\adjective.json
Saved 797 items with part of speech 'adverb' to pos_data\adverb.json
Saved 33 items with part of speech 'exclamation' to pos_data\exclamation.json
Saved 233 items with part of speech 'preposition' to pos_data\preposition.json
Saved 85 items with part of speech 'determiner' to pos_data\determiner.json
Saved 37 items with part of speech 'modal verb' to pos_data\modal verb.json
Saved 6 items with part of speech 'auxiliary verb' to pos_data\auxiliary verb.json
Saved 101 items with part of speech 'pronoun' to pos_data\pronoun.json
Saved 3611 items with part of speech 'phrase' to pos_data\phrase.json
Saved 728 items with part of speech 'phrasal verb' to pos_data\phrasal verb.json

Total files created: 13


In [11]:
import json
import re
from typing import Dict, List, Optional, Any

def parse_sense_title(sense_title: str) -> Dict[str, Optional[str]]:
    """
    解析sense_title，提取括号前的主标题和括号内的内容
    
    Args:
        sense_title: 例如 "well (HEALTHY)" 或 "well"
        
    Returns:
        dict: {"main_title": "well", "parenthetical": "HEALTHY"}
    """
    match = re.match(r'^([^(]+)(?:\s*\(([^)]+)\))?', sense_title)
    if match:
        return {
            "main_title": match.group(1).strip(),
            "parenthetical": match.group(2).strip() if match.group(2) else None
        }
    return {
        "main_title": sense_title.strip(),
        "parenthetical": None
    }

def convert_word_data(word_item: Dict[str, Any]) -> Dict[str, Any]:
    """
    转换单个词条数据
    
    Args:
        word_item: 原始词条数据
        
    Returns:
        dict: 转换后的词条数据
    """
    # 保留一级字段
    result = {
        "word": word_item.get("word"),
        "guideword": word_item.get("guideword"),
        "level": word_item.get("level"),
        "part_of_speech": word_item.get("part_of_speech"),
        "topic": word_item.get("topic"),
        "details": word_item.get("details"),
        "definition": None
    }
    
    # 如果definitions为空或不存在，直接返回
    definitions = word_item.get("definitions", [])

    if not definitions:
        return result
    result["definitions_len"] = len(definitions)
    # 遍历definitions数组
    for def_group in definitions:
        # 暂存pos_header
        pos_header = def_group.get("pos_header")
        pos_header_position = pos_header.get("position") if pos_header else None
        pos_items_len = len(def_group.get("pos_items", []))

        # 遍历pos_items
        pos_items = def_group.get("pos_items", [])
        for pos_item in pos_items:
            # 解析sense_title
            sense_title = pos_item.get("sense_title", "")
            parsed_title = parse_sense_title(sense_title)
            
            # 进行三项匹配
            # 1. sense_title括号前部分对比word
            word_match = (parsed_title["main_title"].lower() == 
                         word_item.get("word", "").lower())
            
            # 2. sense_title括号内部分对比guideword
            guideword = word_item.get("guideword", "")
            if parsed_title["parenthetical"]:
                guideword_match = (parsed_title["parenthetical"].upper() == 
                                 guideword.upper())
            else:
                # 如果没有括号内容，且guideword也为空，则匹配
                guideword_match = not guideword
            
            # 3. level对比level
            level_match = pos_item.get("level") == word_item.get("level")
            
            # 如果三项都匹配成功
            if word_match and guideword_match and level_match:
                # result["definition"] = {
                #     "pos_header": pos_header,
                #     "pos_item": pos_item
                # }
                result["headword"] = pos_header["headword"] if pos_header.get("headword") else None
                result["position"] = pos_header["position"] if pos_header.get("position") else None
                result["definition"] = pos_item["definition"] if pos_item.get("definition") else None
                result["dictionary_examples"] = " | ".join(pos_item.get("dictionary_examples", []))
                result["learner_examples"] = " | ".join(pos_item.get("learner_examples", []))
                return result  # 找到匹配项后立即返回
            elif word_match and pos_header_position == word_item.get("part_of_speech") and level_match and not parsed_title["parenthetical"]:
                result["headword"] = pos_header["headword"] if pos_header.get("headword") else None
                result["position"] = pos_header["position"] if pos_header.get("position") else None
                result["definition"] = pos_item["definition"] if pos_item.get("definition") else None
                result["dictionary_examples"] = " | ".join(pos_item.get("dictionary_examples", []))
                result["learner_examples"] = " | ".join(pos_item.get("learner_examples", []))
                return result

    # 如果没有找到匹配项，definition保持为None
    return result

def convert_word_list(word_list: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    批量转换词条列表
    
    Args:
        word_list: 原始词条列表
        
    Returns:
        list: 转换后的词条列表
    """
    return [convert_word_data(word_item) for word_item in word_list]

def load_and_convert_json(input_file: str, output_file: str = None) -> List[Dict[str, Any]]:
    """
    从JSON文件加载数据并转换
    
    Args:
        input_file: 输入JSON文件路径
        output_file: 输出JSON文件路径（可选）
        
    Returns:
        list: 转换后的数据列表
    """
    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # 如果数据是单个词条，转换为列表
        if isinstance(data, dict):
            data = [data]
        
        # 转换数据
        converted_data = convert_word_list(data)
        
        # 如果指定了输出文件，保存结果
        if output_file:
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(converted_data, f, ensure_ascii=False, indent=2)
            print(f"转换完成，结果已保存到 {output_file}")
        
        return converted_data
        
    except FileNotFoundError:
        print(f"文件 {input_file} 不存在")
        return []
    except json.JSONDecodeError as e:
        print(f"JSON格式错误: {e}")
        return []
    except Exception as e:
        print(f"处理过程中发生错误: {e}")
        return []

# 测试用例
def test_conversion():
    """测试转换功能"""
    test_data = {
        "word": "well",
        "guideword": "HEALTHY",
        "level": "A1",
        "part_of_speech": "adjective",
        "topic": "body and health",
        "details": "/british-english/words/detail/6606",
        "definitions": [
            {
                "pos_header": {
                    "headword": "well",
                    "position": "adverb",
                    "audio": "http://www.englishprofile.org/evp/audio/UKWELDE004.mp3",
                    "written": "/wel/"
                },
                "pos_items": [
                    {
                        "sense_title": "well (IN A GOOD WAY)",
                        "level": "A1",
                        "definition": "in a successful or satisfactory way",
                        "dictionary_examples": [
                            "I thought the team played well.",
                            "He's doing very well at school."
                        ]
                    }
                ]
            },
            {
                "pos_header": {
                    "headword": "well",
                    "position": "adjective",
                    "audio": "http://www.englishprofile.org/evp/audio/UKWELDE004.mp3",
                    "written": "/wel/"
                },
                "pos_items": [
                    {
                        "sense_title": "well (HEALTHY)",
                        "level": "A1",
                        "definition": "healthy; not ill",
                        "dictionary_examples": [
                            "You look well! - Thanks, I feel well.",
                            "He hasn't been too well lately."
                        ]
                    }
                ]
            }
        ]
    }
    
    print("原始数据:")
    print(json.dumps(test_data, indent=2, ensure_ascii=False))
    
    print("\n转换后数据:")
    converted = convert_word_data(test_data)
    print(json.dumps(converted, indent=2, ensure_ascii=False))

# if __name__ == "__main__":
#     # 运行测试
#     test_conversion()
    
    # 使用示例：
    # 1. 转换单个JSON文件
    # converted_data = load_and_convert_json("input.json", "output.json")
    
    # 2. 只转换不保存文件
    # converted_data = load_and_convert_json("input.json")
    
    # 3. 直接使用转换函数
    # with open("your_data.json", "r", encoding="utf-8") as f:
    #     data = json.load(f)
    # converted = convert_word_list(data)

In [12]:
test_conversion()

原始数据:
{
  "word": "well",
  "guideword": "HEALTHY",
  "level": "A1",
  "part_of_speech": "adjective",
  "topic": "body and health",
  "details": "/british-english/words/detail/6606",
  "definitions": [
    {
      "pos_header": {
        "headword": "well",
        "position": "adverb",
        "audio": "http://www.englishprofile.org/evp/audio/UKWELDE004.mp3",
        "written": "/wel/"
      },
      "pos_items": [
        {
          "sense_title": "well (IN A GOOD WAY)",
          "level": "A1",
          "definition": "in a successful or satisfactory way",
          "dictionary_examples": [
            "I thought the team played well.",
            "He's doing very well at school."
          ]
        }
      ]
    },
    {
      "pos_header": {
        "headword": "well",
        "position": "adjective",
        "audio": "http://www.englishprofile.org/evp/audio/UKWELDE004.mp3",
        "written": "/wel/"
      },
      "pos_items": [
        {
          "sense_title": "well (HEALT

In [7]:
len(evp_data)

15696

In [8]:
test_data = convert_word_data(evp_data[0])

In [9]:
print(json.dumps(test_data, ensure_ascii=False, indent=2))

{
  "word": "cattle",
  "guideword": "",
  "level": "B1",
  "part_of_speech": "",
  "topic": "animals",
  "details": "/british-english/words/detail/817",
  "definition": "male and female cows kept on a farm for their milk or meat",
  "definitions_len": 1,
  "headword": "cattle",
  "position": "noun",
  "dictionary_examples": " beef/dairy cattle",
  "learner_examples": "He was born in Australia, where his father has a big cattle station. (Preliminary English Test; B1; German)"
}


In [13]:
converted_data_list = [] 
error_converted_data_list = []
for word_item in evp_data:
    converted = convert_word_data(word_item)
    if converted["definition"]:
        converted_data_list.append(converted)
    else:
        error_converted_data_list.append(converted)


In [14]:
len(converted_data_list)

14189

In [15]:
len(error_converted_data_list)

1507

In [16]:
print(json.dumps(error_converted_data_list[:15], ensure_ascii=False, indent=2))

[
  {
    "word": "forty",
    "guideword": "NUMBER",
    "level": "A2",
    "part_of_speech": "",
    "topic": "",
    "details": "/british-english/words/detail/2359",
    "definition": null,
    "definitions_len": 1
  },
  {
    "word": "hundred",
    "guideword": "NUMBER",
    "level": "A2",
    "part_of_speech": "",
    "topic": "",
    "details": "/british-english/words/detail/2814",
    "definition": null,
    "definitions_len": 2
  },
  {
    "word": "million",
    "guideword": "NUMBER",
    "level": "A2",
    "part_of_speech": "",
    "topic": "",
    "details": "/british-english/words/detail/3625",
    "definition": null,
    "definitions_len": 2
  },
  {
    "word": "ninety",
    "guideword": "NUMBER",
    "level": "A2",
    "part_of_speech": "",
    "topic": "",
    "details": "/british-english/words/detail/3849",
    "definition": null,
    "definitions_len": 1
  },
  {
    "word": "provided (that)",
    "guideword": "",
    "level": "B2",
    "part_of_speech": "",
    "top

In [96]:
import json

# Save the successfully converted data to a JSON file
with open('converted_data.json', 'w', encoding='utf-8') as file:
  json.dump(converted_data_list, file, ensure_ascii=False, indent=2)
  
# Save the data with conversion errors to a separate JSON file
with open('error_converted_data.json', 'w', encoding='utf-8') as file:
  json.dump(error_converted_data_list, file, ensure_ascii=False, indent=2)

print(f"Successfully saved {len(converted_data_list)} successfully converted items to 'converted_data.json'")
print(f"Successfully saved {len(error_converted_data_list)} items with conversion errors to 'error_converted_data.json'")

Successfully saved 13691 successfully converted items to 'converted_data.json'
Successfully saved 2005 items with conversion errors to 'error_converted_data.json'


In [100]:
no_part_of_speech = [item for item in converted_data_list if item['part_of_speech'] == ""]


In [102]:
print(json.dumps(no_part_of_speech, indent=2))

[
  {
    "word": "cattle",
    "guideword": "",
    "level": "B1",
    "part_of_speech": "",
    "topic": "animals",
    "details": "/british-english/words/detail/817",
    "definition": "male and female cows kept on a farm for their milk or meat",
    "definitions_len": 1,
    "headword": "cattle",
    "position": "noun",
    "dictionary_examples": " beef/dairy cattle",
    "learner_examples": "He was born in Australia, where his father has a big cattle station. (Preliminary English Test; B1; German)"
  },
  {
    "word": "clothes",
    "guideword": "",
    "level": "A1",
    "part_of_speech": "",
    "topic": "clothes",
    "details": "/british-english/words/detail/972",
    "definition": "items such as shirts, dresses and trousers that you wear on your body",
    "definitions_len": 1,
    "headword": "clothes",
    "position": "noun",
    "dictionary_examples": "She wears smart/casual clothes. | I'm just putting my clothes on. |  Take your clothes off and get in the bath, Holly. | 

In [92]:
len(converted_data_list), len(error_converted_data_list)

(13691, 2005)

In [93]:
in_defs = [item for item in converted_data_list if item.get("word") == "in"]

In [94]:
len(in_defs)

11

In [95]:
print(json.dumps(in_defs, indent=2, ensure_ascii=False))

[
  {
    "word": "in",
    "guideword": "WEARING",
    "level": "B1",
    "part_of_speech": "preposition",
    "topic": "clothes",
    "details": "/british-english/words/detail/2916",
    "definition": "wearing",
    "definitions_len": 3,
    "headword": "in",
    "position": "preposition",
    "dictionary_examples": "Do you recognize that man in the grey suit? | She's always dressed in red.",
    "learner_examples": "[T]he man has to be dressed in black. (Preliminary English Test; B1; Spanish)"
  },
  {
    "word": "in",
    "guideword": "EXPRESSED",
    "level": "B1",
    "part_of_speech": "preposition",
    "topic": "communication",
    "details": "/british-english/words/detail/2916",
    "definition": "expressed or written in a particular way",
    "definitions_len": 3,
    "headword": "in",
    "position": "preposition",
    "dictionary_examples": "They spoke in Russian the whole time. | Why not do a rough answer in pencil first? | Complete the form in (black) ink.",
    "learner

In [87]:
no_definitions_items = [item for item in error_converted_data_list if item.get("definitions_len", 0) == 0]

In [88]:
len(no_definitions_items)

49

In [89]:
print(json.dumps(no_definitions_items, indent=2, ensure_ascii=False))

[
  {
    "word": "in",
    "guideword": "INTO A SPACE",
    "level": "A2",
    "part_of_speech": "adverb",
    "topic": "describing things",
    "details": "/british-english/words/detail/",
    "definition": null
  },
  {
    "word": "in",
    "guideword": "TRANSPORT",
    "level": "B2",
    "part_of_speech": "adverb",
    "topic": "travel",
    "details": "/british-english/words/detail/",
    "definition": null
  },
  {
    "word": "in",
    "guideword": "AT A PLACE",
    "level": "B1",
    "part_of_speech": "adverb",
    "topic": "",
    "details": "/british-english/words/detail/",
    "definition": null
  },
  {
    "word": "in",
    "guideword": "SENT",
    "level": "B2",
    "part_of_speech": "adverb",
    "topic": "",
    "details": "/british-english/words/detail/",
    "definition": null
  },
  {
    "word": "in the vicinity (of sth)",
    "guideword": "",
    "level": "C2",
    "part_of_speech": "phrase",
    "topic": "",
    "details": "/british-english/words/detail/",
    "d

In [70]:
# print(json.dumps(converted_data_list[:15], indent=2, ensure_ascii=False))
print(json.dumps(error_converted_data_list[:15], indent=2, ensure_ascii=False))

[
  {
    "word": "forty",
    "guideword": "NUMBER",
    "level": "A2",
    "part_of_speech": "",
    "topic": "",
    "details": "/british-english/words/detail/2359",
    "definition": null
  },
  {
    "word": "hundred",
    "guideword": "NUMBER",
    "level": "A2",
    "part_of_speech": "",
    "topic": "",
    "details": "/british-english/words/detail/2814",
    "definition": null
  },
  {
    "word": "million",
    "guideword": "NUMBER",
    "level": "A2",
    "part_of_speech": "",
    "topic": "",
    "details": "/british-english/words/detail/3625",
    "definition": null
  },
  {
    "word": "ninety",
    "guideword": "NUMBER",
    "level": "A2",
    "part_of_speech": "",
    "topic": "",
    "details": "/british-english/words/detail/3849",
    "definition": null
  },
  {
    "word": "provided (that)",
    "guideword": "",
    "level": "B2",
    "part_of_speech": "",
    "topic": "communication",
    "details": "/british-english/words/detail/4565",
    "definition": null
  },
 

In [9]:
from random import randint
def get_user_agent_list():
    response = requests.get(
        'http://headers.scrapeops.io/v1/user-agents?api_key=' + os.environ["SCRAPEOPS_API_KEY"])
    json_response = response.json()
    return json_response.get('result', [])


def get_random_user_agent(user_agent_list):
    random_index = randint(0, len(user_agent_list) - 1)
    return user_agent_list[random_index]


# Retrieve User-Agent List From ScrapeOps
user_agent_list = get_user_agent_list()

In [10]:
import json
# Read the phrasal verb list from the saved JSON file
with open('evp_british_english_pv_list.json', 'r', encoding='utf-8') as f:
  pv_list = json.load(f)

print(f"Loaded {len(pv_list)} phrasal verbs from evp_british_english_pv_list.json")

Loaded 728 phrasal verbs from evp_british_english_pv_list.json


In [18]:

test_item = pv_list[0]
test_url = test_item['details']


In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# 配置Chrome选项
options = Options()
options.add_argument('--headless')  # 无头模式
driver = webdriver.Chrome(options=options)

try:
    driver.get('https://englishprofile.org/?menu=evp-online&refid=ID_00000425')
    # 等待页面加载完成
    driver.implicitly_wait(10)
    html_content = driver.page_source
    print(html_content)
finally:
    driver.quit()

WebDriverException: Message: unknown error: net::ERR_CONNECTION_CLOSED
  (Session info: chrome=139.0.7258.128)
Stacktrace:
	GetHandleVerifier [0x0x7ff64e8478d5+2802741]
	GetHandleVerifier [0x0x7ff64e5aeb70+79568]
	(No symbol) [0x0x7ff64e34c0fa]
	(No symbol) [0x0x7ff64e348ea0]
	(No symbol) [0x0x7ff64e339cc9]
	(No symbol) [0x0x7ff64e33ba81]
	(No symbol) [0x0x7ff64e339fe6]
	(No symbol) [0x0x7ff64e339a46]
	(No symbol) [0x0x7ff64e33970a]
	(No symbol) [0x0x7ff64e3372db]
	(No symbol) [0x0x7ff64e337b5c]
	(No symbol) [0x0x7ff64e3500aa]
	(No symbol) [0x0x7ff64e3f3bce]
	(No symbol) [0x0x7ff64e3cac8a]
	(No symbol) [0x0x7ff64e3f2e24]
	(No symbol) [0x0x7ff64e3caa63]
	(No symbol) [0x0x7ff64e393c91]
	(No symbol) [0x0x7ff64e394a23]
	GetHandleVerifier [0x0x7ff64e872ced+2979917]
	GetHandleVerifier [0x0x7ff64e86d0f3+2956371]
	GetHandleVerifier [0x0x7ff64e88acbd+3078173]
	GetHandleVerifier [0x0x7ff64e5c836e+184014]
	GetHandleVerifier [0x0x7ff64e5d024f+216495]
	GetHandleVerifier [0x0x7ff64e5b70c4+113700]
	G

In [17]:
print(response.text)

<!doctype html>
<html lang="en">
<head>
<meta charset="utf-8">

<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">

<title>English Profile - EVP Online</title>
<script type="text/javascript">
    window.bubble_session_uid = '1755758859569x308418276917804350';
</script>
<script type="text/javascript">
    window.headers_source_maps = [["plugin_main_headers_1696322224077x589893896350203900",215,220]]
</script>
<script type="text/javascript">
function make_proxy(target, name) {
  return new Proxy(target, {
    get: function(target, prop) {
      if (prop in target) {
        return target[prop];
      }
      throw new Error('The variable ' + name + ' is not fully initialized yet');
    }
  });
}
window.appquery = make_proxy({
  app_version: function() { return "live"; },
  last_change: function() { return "24474345191";},
}, 'appquery');
window.Lib = new Proxy(function() {}, {
  get: function(target, prop) {
    if (prop === 'is_skeleton') {
      return true;
    }
    throw

In [16]:
# Search for div elements with class "bubble-element RepeatingGroup baTaJaXd bubble-rg"
repeating_groups = soup.find_all('div', class_="bubble-element")


print(f"Found {len(repeating_groups)} repeating groups")

# If found, examine the first one
if repeating_groups:
  print("First repeating group structure:")
  print(repeating_groups[0].prettify()[:1000])  # Show first 1000 characters
else:
  print("No repeating groups found with that class")

Found 0 repeating groups
No repeating groups found with that class


In [20]:
import requests
from bs4 import BeautifulSoup
from random import randint
from collections import defaultdict
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

CAMBRIDGE_URL = "https://dictionary.cambridge.org"
CAMBRIDGE_CN_SEARCH_URL = CAMBRIDGE_URL + "/search/direct/?datasetsearch=english-chinese-simplified&q="
word_test_block = None
pv_test_block = None
idiom_test_block = None

CAMBRIDGE_EN_SEARCH_URL = CAMBRIDGE_URL + "/search/direct/?datasetsearch=english&q="



# 初始化Chrome浏览器

url="https://englishprofile.org/?menu=evp-online"
# 使用Service类指定ChromeDriver的路径
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get(url)
# 等待页面加载并选择第5个下拉框
dropdowns = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "select.bubble-element.Dropdown.dropdown-chevron"))
)

pos_select = Select(dropdowns[1])
pos_select.select_by_visible_text("phrasal verb")

# 第5个下拉框（索引4）
# select = Select(dropdowns[4])
# select.select_by_visible_text("All")

# print("第5个下拉框已设置为All")
time.sleep(10)


# html = driver.page_source

# headers = {'User-Agent': get_random_user_agent(user_agent_list)}
# All words data list to store the results
# all_words_data = []

# # Function to extract word data from the current page
# def extract_word_data_from_page():
#     # Wait for the word entries to load
#     wait = WebDriverWait(driver, 20)  # 20 seconds timeout
#     wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.bubble-element.RepeatingGroup.baTaJaLu')))
#     wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.bubble-element.group-item.bubble-r-container.flex.column')))
    
#     # Add a short delay to ensure all content is loaded
#     time.sleep(2)
    
#     # Get the page source and parse with BeautifulSoup
#     soup = BeautifulSoup(driver.page_source, 'html.parser')
    
#     # Find all word entries
#     word_entries = soup.select('div.bubble-element.RepeatingGroup.baTaJaLu div.bubble-element.group-item.bubble-r-container.flex.column')
    
#     page_data = []
#     for entry in word_entries:
#         # Extract information for each field
#         base_word_element = entry.select_one('div.baTaJaLaW')
#         guideword_element = entry.select_one('div.baTaJaLaX')
#         level_element = entry.select_one('div.baTaJaLb')
#         pos_element = entry.select_one('div.baTaJaQaI')
#         topic_element = entry.select_one('div.baTaJaQaO')
#         details_button = entry.select_one('button.baTaJaLd')
#         details_url = None
        
#         if details_button:
#             # Extract details URL if available
#             onclick_attr = details_button.get('onclick', '')
#             url_match = re.search(r"'(https://englishprofile\.org/british-english/words/detail/\d+)'", onclick_attr)
#             if url_match:
#                 details_url = url_match.group(1)
        
#         # Extract text content
#         base_word = base_word_element.get_text(strip=True) if base_word_element else ''
#         guideword = guideword_element.get_text(strip=True) if guideword_element else ''
#         level = level_element.get_text(strip=True) if level_element else ''
#         part_of_speech = pos_element.get_text(strip=True) if pos_element else ''
#         topic = topic_element.get_text(strip=True) if topic_element else ''
        
#         # Create a dictionary for the current word
#         word_data = {
#             'word': base_word,
#             'guideword': guideword,
#             'level': level,
#             'part_of_speech': part_of_speech,
#             'topic': topic,
#             'details': details_url
#         }
#         page_data.append(word_data)
    
#     return page_data

# Total number of pages to scrape
total_pages = 37  # From your existing code
items_per_page = 20  # Number of items per page, adjust if necessary
# Scrape all pages
try:
    # Process current page (first page)
    # current_page_data = extract_word_data_from_page()
    # all_words_data.extend(current_page_data)
    # print(f"Scraped page 1 with {len(current_page_data)} entries")
    details_buttons = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'clickable-element bubble-element Button baTaJaLd flex-aligned-button')))
    print(len(details_buttons))
    
    # Navigate through remaining pages
    for page_num in range(2, total_pages + 1):
        try:
            # Find and click the next page button
            wait = WebDriverWait(driver, 20)
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.baTaJaMaB')))
            driver.execute_script("arguments[0].click();", next_button)

            details_buttons = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'clickable-element bubble-element Button baTaJaLd flex-aligned-button')))
            print(len(details_buttons))
            # Wait for page to load and extract data
            time.sleep(2)  # Give time for the page to load
            # current_page_data = extract_word_data_from_page()
            # all_words_data.extend(current_page_data)
            
            print(f"Scraped page {page_num} with {len(current_page_data)} entries")
            
            
        except Exception as e:
            print(f"Error on page {page_num}: {e}")
            break
            
except Exception as e:
    print(f"An error occurred during scraping: {e}")

# finally:
    # Save the data to a JSON file
    # with open('evp_phrasal_verbs_list.json', 'w', encoding='utf-8') as f:
    #     json.dump(all_words_data, f, ensure_ascii=False, indent=2)
    # print(f"Successfully saved {len(all_words_data)} phrasal verbs to evp_phrasal_verbs_list.json")
    
    # Don't close the driver here as you might want to continue using it
    # driver.quit()




WebDriverException: Message: unknown error: net::ERR_CONNECTION_CLOSED
  (Session info: chrome=139.0.7258.128)
Stacktrace:
	GetHandleVerifier [0x0xeaffc3+65331]
	GetHandleVerifier [0x0xeb0004+65396]
	(No symbol) [0x0xca3f63]
	(No symbol) [0x0xca16cb]
	(No symbol) [0x0xc94e82]
	(No symbol) [0x0xc968b8]
	(No symbol) [0x0xc95118]
	(No symbol) [0x0xc94c53]
	(No symbol) [0x0xc94961]
	(No symbol) [0x0xc92904]
	(No symbol) [0x0xc932ab]
	(No symbol) [0x0xca797e]
	(No symbol) [0x0xd337e7]
	(No symbol) [0x0xd114dc]
	(No symbol) [0x0xd32bcb]
	(No symbol) [0x0xd112d6]
	(No symbol) [0x0xce0910]
	(No symbol) [0x0xce1784]
	GetHandleVerifier [0x0x10f38b3+2439203]
	GetHandleVerifier [0x0x10eeae2+2419282]
	GetHandleVerifier [0x0xed712a+225434]
	GetHandleVerifier [0x0xec6e08+159096]
	GetHandleVerifier [0x0xecdd5d+187597]
	GetHandleVerifier [0x0xeb7ad8+96840]
	GetHandleVerifier [0x0xeb7c62+97234]
	GetHandleVerifier [0x0xea277a+9962]
	BaseThreadInitThunk [0x0x7665fcc9+25]
	RtlGetAppContainerNamedObjectPath

In [ ]:
all_words_data = []

# 网站总共有157页
total_pages = 37

if page_num < total_pages:
    try:
        # 使用WebDriverWait等待数据加载完成
        wait = WebDriverWait(driver, 20)  # 最多等待20秒
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.bubble-element.RepeatingGroup.baTaJaLu')))
                
        # 等待至少有一个词汇条目加载出来
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.bubble-element.group-item.bubble-r-container.flex.column')))

        # 增加一个短暂的固定延时，确保所有动态内容都已渲染
        time.sleep(2)
        # 找到并点击“向右”的箭头按钮来翻页
        next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.baTaJaMaB')))
        driver.execute_script("arguments[0].click();", next_button)
    except Exception as e:
        print(f"在第 {page_num} 页无法找到或点击下一页按钮: {e}")
        # break # 如果找不到按钮，则退出循环
# 获取页面源代码并用BeautifulSoup解析
# soup = BeautifulSoup(driver.page_source, 'html.parser')
# # 查找所有词汇条目
# # 每个词条都在一个特定的div容器里
# word_entries = soup.select('div.bubble-element.RepeatingGroup.baTaJaLu div.bubble-element.group-item.bubble-r-container.flex.column')
# if not word_entries:
#     print(f"警告：在第 {page_num} 页未找到词汇条目。")

#             # 遍历每个词汇条目并提取信息
# for entry in word_entries:
#     # 使用select_one确保即使某个字段不存在也不会报错，而是返回None
#     base_word_element = entry.select_one('div.baTaJaLaW')
#     guideword_element = entry.select_one('div.baTaJaLaX')
#     level_element = entry.select_one('div.baTaJaLb')
#     pos_element = entry.select_one('div.baTaJaQaI')
#     topic_element = entry.select_one('div.baTaJaQaO')
#     details_button = entry.select_one('button.baTaJaLd')
#     # 提取文本内容，如果元素不存在则返回空字符串
#     base_word = base_word_element.get_text(strip=True) if base_word_element else ''
#     guideword = guideword_element.get_text(strip=True) if guideword_element else ''
#     level = level_element.get_text(strip=True) if level_element else ''
#     part_of_speech = pos_element.get_text(strip=True) if pos_element else ''
#     topic = topic_element.get_text(strip=True) if topic_element else ''                
#     # 对于Details链接，我们只记录它是否存在
#     has_details_link = '是' if details_button else '否'
#     # 将提取的数据存入字典
#     word_data = {
#         'Base word': base_word,
#         'Guideword': guideword,
#         'Level': level,
#         'Part of Speech': part_of_speech,
#         'Topic': topic,
#         'Details Link Available': has_details_link
#     }
#     all_words_data.append(word_data)

SyntaxError: 'break' outside loop (3740401200.py, line 22)

In [36]:
len(all_words_data)

244

In [ ]:


    try:
        for page_num in range(1, total_pages + 1):
            print(f"正在抓取第 {page_num} / {total_pages} 页...")

            # 使用WebDriverWait等待数据加载完成
            # 等待包含词汇列表的容器元素出现
            wait = WebDriverWait(driver, 20) # 最多等待20秒
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.bubble-element.RepeatingGroup.baTaJaLu')))
            
            # 等待至少有一个词汇条目加载出来
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.bubble-element.group-item.bubble-r-container.flex.column')))

            # 增加一个短暂的固定延时，确保所有动态内容都已渲染
            time.sleep(2)

            # 获取页面源代码并用BeautifulSoup解析
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # 查找所有词汇条目
            # 每个词条都在一个特定的div容器里
            word_entries = soup.select('div.bubble-element.RepeatingGroup.baTaJaLu div.bubble-element.group-item.bubble-r-container.flex.column')

            if not word_entries:
                print(f"警告：在第 {page_num} 页未找到词汇条目。")
                break

            # 遍历每个词汇条目并提取信息
            for entry in word_entries:
                # 使用select_one确保即使某个字段不存在也不会报错，而是返回None
                base_word_element = entry.select_one('div.baTaJaLaW')
                guideword_element = entry.select_one('div.baTaJaLaX')
                level_element = entry.select_one('div.baTaJaLb')
                pos_element = entry.select_one('div.baTaJaQaI')
                topic_element = entry.select_one('div.baTaJaQaO')
                details_button = entry.select_one('button.baTaJaLd')

                # 提取文本内容，如果元素不存在则返回空字符串
                base_word = base_word_element.get_text(strip=True) if base_word_element else ''
                guideword = guideword_element.get_text(strip=True) if guideword_element else ''
                level = level_element.get_text(strip=True) if level_element else ''
                part_of_speech = pos_element.get_text(strip=True) if pos_element else ''
                topic = topic_element.get_text(strip=True) if topic_element else ''
                
                # 对于Details链接，我们只记录它是否存在
                has_details_link = '是' if details_button else '否'

                # 将提取的数据存入字典
                word_data = {
                    'Base word': base_word,
                    'Guideword': guideword,
                    'Level': level,
                    'Part of Speech': part_of_speech,
                    'Topic': topic,
                    'Details Link Available': has_details_link
                }
                all_words_data.append(word_data)

            # 如果不是最后一页，则点击下一页按钮
            if page_num < total_pages:
                try:
                    # 找到并点击“向右”的箭头按钮来翻页
                    next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.baTaJaMaB')))
                    driver.execute_script("arguments[0].click();", next_button)
                except Exception as e:
                    print(f"在第 {page_num} 页无法找到或点击下一页按钮: {e}")
                    break # 如果找不到按钮，则退出循环
    
    except Exception as e:
        print(f"抓取过程中发生错误: {e}")

    finally:
        # 关闭浏览器
        driver.quit()

    # 将数据转换为Pandas DataFrame并保存为CSV文件
    if all_words_data:
        df = pd.DataFrame(all_words_data)
        # 使用 utf-8-sig 编码以确保在Excel中打开时中文字符不会乱码
        df.to_csv('vocabulary_data.csv', index=False, encoding='utf-8-sig')
        print(f"\n抓取完成！共抓取 {len(all_words_data)} 条数据，已保存至 vocabulary_data.csv 文件。")
    else:
        print("\n未能抓取到任何数据。")

In [21]:
print(html)

<html lang="en" class="wf-raleway-n3-active wf-raleway-n4-active wf-raleway-n6-active wf-raleway-n7-active wf-active" style="height: 100%; --font_default: &quot;Raleway&quot;; --color_text_default: rgba(34,34,34,1); --color_text_default_rgb: 34,34,34; --color_alert_default: rgba(220,161,20,1); --color_alert_default_rgb: 220,161,20; --color_primary_default: rgba(0,85,140,1); --color_primary_default_rgb: 0,85,140; --color_success_default: rgba(30,108,48,1); --color_success_default_rgb: 30,108,48; --color_surface_default: rgba(255,255,255,1); --color_surface_default_rgb: 255,255,255; --color_background_default: rgba(255, 255, 255, 1); --color_background_default_rgb: 255,255,255; --color_destructive_default: rgba(176,32,12,1); --color_destructive_default_rgb: 176,32,12; --color_primary_contrast_default: rgba(255,255,255,1); --color_primary_contrast_default_rgb: 255,255,255; --color_bTGzp_default: rgba(245,250,255,1); --color_bTGzp_default_rgb: 245,250,255; --color_bTGzq_default: rgba(235,2

In [34]:
# Save the HTML content to a file
with open('evp.html', 'w', encoding='utf-8') as f:
  f.write(html)

print("HTML content saved to evp.html")

HTML content saved to evp.html


In [22]:
soup = BeautifulSoup(html, 'html.parser')

  # Find all word items in the EVP list
word_items = soup.find_all('div', class_=lambda c: c and c.startswith('bubble-element group-item bubble-r-container flex column'))

  


In [23]:
len(word_items)

152

In [9]:
evp_data = get_evp()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=139.0.7258.128)
Stacktrace:
	GetHandleVerifier [0x0x2afd33+62915]
	GetHandleVerifier [0x0x2afd74+62980]
	(No symbol) [0x0xe3e13]
	(No symbol) [0x0xc2c29]
	(No symbol) [0x0x157ade]
	(No symbol) [0x0x171fc9]
	(No symbol) [0x0x1511d6]
	(No symbol) [0x0x120833]
	(No symbol) [0x0x1216a4]
	GetHandleVerifier [0x0x518d23+2590131]
	GetHandleVerifier [0x0x513f6a+2570234]
	GetHandleVerifier [0x0x2d59ea+217722]
	GetHandleVerifier [0x0x2c6058+153832]
	GetHandleVerifier [0x0x2cc4bd+179533]
	GetHandleVerifier [0x0x2b7738+94152]
	GetHandleVerifier [0x0x2b78c2+94546]
	GetHandleVerifier [0x0x2a2bda+9322]
	BaseThreadInitThunk [0x0x7665fcc9+25]
	RtlGetAppContainerNamedObjectPath [0x0x77bb82ae+286]
	RtlGetAppContainerNamedObjectPath [0x0x77bb827e+238]
	(No symbol) [0x0]


In [30]:
print(evp_data)

[]


In [27]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 初始化WebDriver
driver = webdriver.Chrome()

# 打开目标网页
driver.get("https://englishprofile.org/?menu=evp-online")  # 根据截图推测的URL

# 等待页面加载并选择第5个下拉框
dropdowns = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "select.bubble-element.Dropdown.dropdown-chevron"))
)

# 第5个下拉框（索引4）
select = Select(dropdowns[4])
select.select_by_visible_text("All")

print("第5个下拉框已设置为All")

第5个下拉框已设置为All


NameError: name 'WebDriverWait' is not defined

In [ ]:
import requests
from bs4 import BeautifulSoup
from random import randint
from collections import defaultdict
import re

CAMBRIDGE_URL = "https://dictionary.cambridge.org"
CAMBRIDGE_CN_SEARCH_URL = CAMBRIDGE_URL + "/search/direct/?datasetsearch=english-chinese-simplified&q="
word_test_block = None
pv_test_block = None
idiom_test_block = None

def get_definition0(word,query_type="word"):
  definition_dict = dict()
  url = CAMBRIDGE_CN_SEARCH_URL + word
  definition_dict["query_word"] = word
  definition_dict["query_type"] = query_type
  definition_dict["returned_type"] = set()
  headers = {'User-Agent': get_random_user_agent(user_agent_list)}


  # 发送HTTP请求获取网页内容
  try:
    response = requests.get(url=url, headers=headers)
  except requests.RequestException as e:
    definition_dict['result'] = "error"
    return definition_dict

  # 使用BeautifulSoup解析HTML内容
  # 获取response中的返回url
  definition_dict["response_url"] = response.url

  # 判断response.url的倒数第二个字段是否是“english-chinese-simplified”
  url_parts = response.url.split('/')
  if len(url_parts) > 2 and url_parts[-2] == "english-chinese-simplified":
    # 判断response.url中的最后一个字段，如果其中包含?q=，则表示查询的词与返回的词不完全一致
    last_url_part = url_parts[-1]
    if "?q=" in last_url_part:
      returned_word = last_url_part.split("?q=")[0]
      definition_dict["returned_word"] = returned_word
    else:
      definition_dict["returned_word"] = last_url_part
  else:
    # 表示双解词典内没有该单词的释义，直接返回
    definition_dict["result"] = "redirected to english dictionary"
    return definition_dict

  soup = BeautifulSoup(response.text, 'html.parser')

  # 通过查找title元素的内容来判断是否成功获取了单词的释义
  title_element = soup.head.find('title', string=lambda t: "Did you spell it correctly?" in t)
  if title_element:
    definition_dict['result'] = "not found"
    return definition_dict
  else:
    definition_dict['result'] = "found"

  # 如果是english dictionary，会有一个或多个pr dictionary的div元素存在，每个div元素表示一个dictionary
  # 第一个pr dictionary的div元素表示English–Mandarin Chinese dictionary，我们忽略其他的dictionary
  # 然后再在期内搜索class="pr di superentry"的元素
  # dictionary_div = soup.find('div', class_='pr dictionary')
  # superentries = dictionary_div.find_all('div', class_='pr di superentry') if dictionary_div else []

  # 如果是english-chinese-simplified dictionary，疑似只有一个pr di superentry的div元素存在
  # 查找class="pr di superentry"的元素
  superentries = soup.find_all(class_="pr di superentry")
  # definition_dict["superentries"] = []
  for superentry in superentries:
    superentry = superentry.find('div', class_='di-body', recursive=False)
    superentry_dict = dict()
    superentry_dict["entries"] = []
    
    # 判断是否有idiom-block元素和entry-body元素（表示存在word或pv）
    idioms_blocks = superentry.find_all('div', class_='pr idiom-block', recursive=False)
    entry_bodies = superentry.select('div.entry > div.entry-body')

    # 存在idiom-block元素，解析idiom-block元素
    if idioms_blocks:
      definition_dict["returned_type"].add("idiom")
      for entry in idioms_blocks:
        # idiom_test_block用于拍错
        idiom_test_block = entry
        entry_dict = dict()
        entry_dict["type"] = "idiom"
        entry_dict["entry_elements"] = []
        idiom_elements = entry.find_all('div', class_='idiom-block', recursive=False)
        for element in idiom_elements:
          # element_dict = "idiom-element"
          entry_dict['entry_elements'].append(parse_idiom_elements(element))
        superentry_dict["entries"].append(entry_dict)
    
    # 存在entry-body元素，解析entry-body元素
    if entry_bodies:
        for entry_body in entry_bodies:
            # 在这里对每个entry_body进行处理
            # 查找class="pr entry-body__el"的元素
            entry_body_dict = dict()

            # 判断是否有word_entry_elements和pv_entry_elements
            word_entry_elements = entry_body.find_all('div', class_='pr entry-body__el', recursive=False)
            # print(f"word_entry_elements length: {len(word_entry_elements)}")
            phrasal_verbs_entry_elements = entry_body.find_all('div', class_='entry-body__el clrd js-share-holder', recursive=False)
            # print(f"phrasal_verbs_entry_elements length: {len(phrasal_verbs_entry_elements)}")
            # 如果有word_entry_elements，解析word_entry_elements
            if len(word_entry_elements) > 0:
              # for i in range(len(word_entry_elements)):
              #   word_entry_elements[i] = "word_element"

              entry_body_dict['entry_elements'] = word_entry_elements
              entry_body_dict["type"] = "word"      
              definition_dict["returned_type"].add("word")
              parse_word_entry_elements(entry_body_dict, word_entry_elements)

            # 如果有pv_entry_elements，解析pv_entry_elements
            if len(phrasal_verbs_entry_elements) > 0:
              # for i in range(len(phrasal_verbs_entry_elements)):
              #   phrasal_verbs_entry_elements[i] = "pv_element"
              entry_body_dict['entry_elements'] = phrasal_verbs_entry_elements
              entry_body_dict["type"] = "phrasal-verb"
              definition_dict["returned_type"].add("phrasal verb")
              parse_phrasal_verb_elements(entry_body_dict,phrasal_verbs_entry_elements)
                  
            superentry_dict["entries"].append(entry_body_dict)
    # definition_dict["superentries"].append(superentry_dict)
    definition_dict["entries"] = superentry_dict["entries"]
  definition_dict["returned_type"] = list(definition_dict["returned_type"])
  return definition_dict
    

In [16]:
import requests
from bs4 import BeautifulSoup
from random import randint
from collections import defaultdict
import re

CAMBRIDGE_URL = "https://dictionary.cambridge.org"
CAMBRIDGE_CN_SEARCH_URL = CAMBRIDGE_URL + "/search/direct/?datasetsearch=english-chinese-simplified&q="
word_test_block = None
pv_test_block = None
idiom_test_block = None

def get_definition(word,query_type="word"):
  definition_dict = dict()
  url = CAMBRIDGE_CN_SEARCH_URL + word
  definition_dict["query_word"] = word
  definition_dict["query_type"] = query_type
  definition_dict["returned_type"] = set()
  headers = {'User-Agent': get_random_user_agent(user_agent_list)}


  # 发送HTTP请求获取网页内容
  try:
    response = requests.get(url=url, headers=headers)
  except requests.RequestException as e:
    definition_dict['result'] = "error"
    return definition_dict

  # 使用BeautifulSoup解析HTML内容
  # 获取response中的返回url
  definition_dict["response_url"] = response.url

  # 判断response.url的倒数第二个字段是否是“english-chinese-simplified”
  url_parts = response.url.split('/')
  if len(url_parts) > 2 and url_parts[-2] == "english-chinese-simplified":
    # 判断response.url中的最后一个字段，如果其中包含?q=，则表示查询的词与返回的词不完全一致
    last_url_part = url_parts[-1]
    if "?q=" in last_url_part:
      returned_word = last_url_part.split("?q=")[0]
      definition_dict["returned_word"] = returned_word
    else:
      definition_dict["returned_word"] = last_url_part
  else:
    # 表示双解词典内没有该单词的释义，直接返回
    definition_dict["result"] = "redirected to english dictionary"
    return definition_dict

  soup = BeautifulSoup(response.text, 'html.parser')

  # 通过查找title元素的内容来判断是否成功获取了单词的释义
  title_element = soup.head.find('title', string=lambda t: "Did you spell it correctly?" in t)
  if title_element:
    definition_dict['result'] = "not found"
    return definition_dict
  else:
    definition_dict['result'] = "found"

  # 如果是english dictionary，会有一个或多个pr dictionary的div元素存在，每个div元素表示一个dictionary
  # 第一个pr dictionary的div元素表示English–Mandarin Chinese dictionary，我们忽略其他的dictionary
  # 然后再在期内搜索class="pr di superentry"的元素
  # dictionary_div = soup.find('div', class_='pr dictionary')
  # superentries = dictionary_div.find_all('div', class_='pr di superentry') if dictionary_div else []

  # 如果是english-chinese-simplified dictionary，疑似只有一个pr di superentry的div元素存在
  # 查找class="pr di superentry"的元素
  superentries = soup.find_all(class_="pr di superentry")
  # definition_dict["superentries"] = []
  for superentry in superentries:
    superentry = superentry.find('div', class_='di-body', recursive=False)
    superentry_dict = dict()
    superentry_dict["entries"] = []
    
    # 判断是否有idiom-block元素和entry-body元素（表示存在word或pv）
    idioms_blocks = superentry.find_all('div', class_='pr idiom-block', recursive=False)
    entry_bodies = superentry.select('div.entry > div.entry-body')

    # 存在idiom-block元素，解析idiom-block元素
    if idioms_blocks:
      definition_dict["returned_type"].add("idiom")
      for entry in idioms_blocks:
        # idiom_test_block用于拍错
        idiom_test_block = entry
        entry_dict = dict()
        entry_dict["type"] = "idiom"
        entry_dict["entry_elements"] = []
        idiom_elements = entry.find_all('div', class_='idiom-block', recursive=False)
        for element in idiom_elements:
          # element_dict = "idiom-element"
          entry_dict['entry_elements'].append(parse_idiom_elements(element))
        superentry_dict["entries"].append(entry_dict)
    
    # 存在entry-body元素，解析entry-body元素
    if entry_bodies:
        for entry_body in entry_bodies:
            # 在这里对每个entry_body进行处理
            # 查找class="pr entry-body__el"的元素
            entry_body_dict = dict()

            # 判断是否有word_entry_elements和pv_entry_elements
            word_entry_elements = entry_body.find_all('div', class_='pr entry-body__el', recursive=False)
            # print(f"word_entry_elements length: {len(word_entry_elements)}")
            phrasal_verbs_entry_elements = entry_body.find_all('div', class_='entry-body__el clrd js-share-holder', recursive=False)
            # print(f"phrasal_verbs_entry_elements length: {len(phrasal_verbs_entry_elements)}")
            # 如果有word_entry_elements，解析word_entry_elements
            if len(word_entry_elements) > 0:
              # for i in range(len(word_entry_elements)):
              #   word_entry_elements[i] = "word_element"

              entry_body_dict['entry_elements'] = word_entry_elements
              entry_body_dict["type"] = "word"      
              definition_dict["returned_type"].add("word")
              parse_word_entry_elements(entry_body_dict, word_entry_elements)

            # 如果有pv_entry_elements，解析pv_entry_elements
            if len(phrasal_verbs_entry_elements) > 0:
              # for i in range(len(phrasal_verbs_entry_elements)):
              #   phrasal_verbs_entry_elements[i] = "pv_element"
              entry_body_dict['entry_elements'] = phrasal_verbs_entry_elements
              entry_body_dict["type"] = "phrasal-verb"
              definition_dict["returned_type"].add("phrasal verb")
              parse_phrasal_verb_elements(entry_body_dict,phrasal_verbs_entry_elements)
                  
            superentry_dict["entries"].append(entry_body_dict)
    # definition_dict["superentries"].append(superentry_dict)
    definition_dict["superentries"] = superentry_dict["entries"]
  definition_dict["returned_type"] = list(definition_dict["returned_type"])
  return definition_dict
    

In [13]:
def parse_word_entry_elements(entry_body_dict,entry_elements):
  entry_body_dict["entry_elements"] = []
  for entry_element in entry_elements:
    # 在这里对每个entry_element进行处理
    entry_element_dict = dict()
    pos_header = entry_element.find('div', class_='pos-header dpos-h')
    if pos_header:
        entry_element_dict["pos_header"] = str(pos_header)
    else:
        entry_element_dict["pos_header"] = None
    pos_body = entry_element.find('div', class_='pos-body')
    entry_element_dict["pos_body"] = dict()
    pos_body_elements = pos_body.find_all('div', class_=re.compile(r'^pr dsense'))
    entry_element_dict["pos_body"]["dsense"] = [str(element) for element in pos_body_elements]
    # pos_body_elements = pos_body.find_all('div', class_='pr dsense dsense-noh')
    # entry_element_dict["pos_body"]["dsense"] = [str(element) for element in pos_body_elements]

    grammar_element = pos_body.find('div', class_='xref grammar hax dxref-w lmt-25 lmb-25')
    if grammar_element:
        entry_element_dict["pos_body"]["grammar"] = str(grammar_element)
    else:
        entry_element_dict["pos_body"]["grammar"] = None

    idioms_element = pos_body.find('div', class_='xref idioms hax dxref-w lmt-25 lmb-25')
    if idioms_element:
        entry_element_dict["pos_body"]["idioms"] = str(idioms_element)
    else:
        entry_element_dict["pos_body"]["idioms"] = None

    phrasal_verbs_element = pos_body.find('div', class_='xref phrasal_verbs hax dxref-w lmt-25 lmb-25')
    if phrasal_verbs_element:
        entry_element_dict["pos_body"]["phrasal_verbs"] = str(phrasal_verbs_element)
    else:
        entry_element_dict["pos_body"]["phrasal_verbs"] = None

    entry_body_dict["entry_elements"].append(entry_element_dict)
  return entry_body_dict

In [14]:
def parse_phrasal_verb_elements(entry_body_dict, entry_elements):
    entry_body_dict["entry_elements"] = []
    for entry_element in entry_elements:
        # 在这里对每个entry_element进行处理
        entry_element_dict = dict()
        entry_element = entry_element.find('div', class_='pr relativDiv').find('div', class_='pv-block')
        
        
        pv_title = entry_element.find('div', class_='di-title')
        if pv_title:
            entry_element_dict["pv_title"] = str(pv_title)
        else:
            entry_element_dict["pv_title"] = None
        pv_info = entry_element.find('span', class_='di-info')
        if pv_info:
            entry_element_dict["pv_info"] = str(pv_info)
        else:
            entry_element_dict["pv_info"] = None
        pv_body = entry_element.find('span', class_='pv-body dpv-body')


        entry_element_dict["pv_body"] = dict()
        pv_body_elements = pv_body.find_all('div', class_=re.compile(r'^pr dsense'))
        
        entry_element_dict["pv_body"]["dsense"] = [str(element) for element in pv_body_elements]
        entry_body_dict["entry_elements"].append(entry_element_dict)
    return entry_body_dict

In [15]:
def parse_idiom_elements(idiom_block):
    idiom_block_dict = dict()
    # idiom_block = idiom_block.find('div', class_='idiom-block')
    idiom_title = idiom_block.find('div', class_='di-title')
    if idiom_title:
        idiom_block_dict["idiom_title"] = str(idiom_title)
    else:
        idiom_block_dict["idiom_title"] = None
    idiom_info = idiom_block.find('span', class_='di-info')
    if idiom_info:
        idiom_block_dict["idiom_info"] = str(idiom_info)
    else:
        idiom_block_dict["idiom_info"] = None
    idiom_body = idiom_block.find('span', class_='idiom-body didiom-body')
    idiom_block_dict["idiom_body"] = dict()
    idiom_body_elements = idiom_body.find_all('div', class_=re.compile(r'^pr dsense'))
    idiom_block_dict["idiom_body"]["dsense"] = [str(element) for element in idiom_body_elements]
    return idiom_block_dict

In [38]:
import json
definition = get_definition("unlock", 'word')

In [54]:
len(definition['entries'][0]['entry_elements'])
print(definition["entries"][0]['entry_elements'][0]['pos_body']['dsense'][0])
# print(definition["entries"][0]["type"])
# print(len(definition["entries"][0]["entry_elements"]))
# print(definition["entries"][0]["entry_elements"][0].keys())
# print(definition["entries"][0]["entry_elements"][0]["pv_title"])
# print(definition["entries"][0]["entry_elements"][0]["pv_info"])
# print(definition.keys())
# print(len(definition["entries"]))
# print(definition["entries"][1].keys())
# print(len(definition["entries"][1]["entry_elements"]))
# print(definition["entries"][1]["entry_elements"][0]["pos_body"]['dsense'][0])

<div class="pr dsense dsense-noh"><div class="cid" id="caldcns-1-1"></div> <div class="sense-body dsense_b"><div class="def-block ddef_block" data-wl-senseid="ID_00034651_01">
<div class="dwl hax">
<a amp-access="loggedIn" amp-access-hide="" class="dwla wordlist-add-button" on="tap:AMP.setState({ stateGlobal: { wlSenseId: 'ID_00034651_01', wotd: '    unlock
' } }), sidebarWordList.open" title="Add this meaning to a word list">
<span class="tb fs10 hvm">Add to word list</span>
<i aria-hidden="true" class="i i-list-ul"></i>
</a>
<a amp-access="NOT loggedIn" class="dwla wordlist-add-button" on="tap:amp-access.login-sign-in" title="Add this meaning to a word list">
<span class="tb fs10 hvm">Add to word list</span>
<i aria-hidden="true" class="i i-list-ul"></i>
</a>
</div>
<div class="ddef_h"><span class="def-info ddef-info"><span class="epp-xref dxref B2">B2</span> </span><div class="def ddef_d db">to <a class="query" href="https://dictionary.cambridge.org/dictionary/english-chinese-simpli

In [137]:
print(definition["entries"][2]["entry_elements"][0]["pos_body"]["dsense"][0])

<div class="pr dsense dsense-noh"><div class="cid" id="caldcns-3-3"></div> <div class="sense-body dsense_b"><div class="def-block ddef_block" data-wl-senseid="ID_00004229_01">
<div class="dwl hax">
<a amp-access="loggedIn" amp-access-hide="" class="dwla wordlist-add-button" on="tap:AMP.setState({ stateGlobal: { wlSenseId: 'ID_00004229_01', wotd: '    burn out
' } }), sidebarWordList.open" title="Add this meaning to a word list">
<i aria-hidden="true" class="i i-plus"></i>
<i aria-hidden="true" class="i i-list-ul"></i>
</a>
<a amp-access="NOT loggedIn" class="dwla wordlist-add-button" on="tap:amp-access.login-sign-in" title="Add this meaning to a word list">
<i aria-hidden="true" class="i i-plus"></i>
<i aria-hidden="true" class="i i-list-ul"></i>
</a>
</div>
<div class="ddef_h"><span class="def-info ddef-info"> </span><div class="def ddef_d db"><a class="query" href="https://dictionary.cambridge.org/dictionary/english-chinese-simplified/extreme" rel="" title="extreme">extreme</a> <a cl

In [6]:
import json

with open('idioms_test_dict.json', 'r', encoding='utf-8') as file:
    idioms_test_dict = json.load(file)
    found_idioms = [key for key, value in idioms_test_dict.items() if value['result'] == 'found']
    print(found_idioms)
    with open('found_idioms.json', 'w', encoding='utf-8') as f:
        json.dump(found_idioms, f, ensure_ascii=False, indent=4)

with open('phrases_test_dict.json', 'r', encoding='utf-8') as file:
    phrases_test_dict = json.load(file)
    found_phrases = [key for key, value in phrases_test_dict.items() if value['result'] == 'found']
    print(found_phrases)
    with open('found_phrases.json', 'w', encoding='utf-8') as f:
        json.dump(found_phrases, f, ensure_ascii=False, indent=4)

with open('pv_test_dict.json', 'r', encoding='utf-8') as file:
    pv_test_dict = json.load(file)
    found_pv = [key for key, value in pv_test_dict.items() if value['result'] == 'found']
    print(found_pv)
    with open('found_pv.json', 'w', encoding='utf-8') as f:
        json.dump(found_pv, f, ensure_ascii=False, indent=4)

with open('words_test_dict.json', 'r', encoding='utf-8') as file:
    words_test_dict = json.load(file)
    found_words = [key for key, value in words_test_dict.items() if value['result'] == 'found']
    print(found_words)
    with open('found_words.json', 'w', encoding='utf-8') as f:
        json.dump(found_words, f, ensure_ascii=False, indent=4)

['a tight corner', 'pave the way', 'know the ropes', "break sb's heart", 'the jury is (still) out', 'slip your mind', 'have a chip on your shoulder', 'pull strings', 'put your feet up', 'a walk down memory lane', 'know your stuff', 'beyond your wildest dreams', 'with bated breath', 'pull your weight', 'blow your mind', 'know sth inside out', 'touch and go', 'play games', 'easier said than done', 'behind closed doors', 'a bumpy ride', 'at face value', "catch sb's eye", 'run riot', 'with flying colours', 'set the record straight', 'throw caution to the wind', 'bring a lump to your throat', 'keep body and soul together', 'go hand in hand', "cross sb's path", 'leave someone to their own devices', 'get your act together', 'cross your mind', 'call it a day', 'a bone of contention']
['in case of sth', 'speak of sth', 'pretty well', 'not say a word', 'not be up to much', 'be meant to do sth', 'keep sth in mind', 'at your convenience', 'take exception to sth', 'apart from', 'regards', 'not leas

In [8]:
found_idioms_list = []
with open('found_idioms.json', 'r', encoding='utf-8') as file:
    found_idioms_list = json.load(file)
print(found_idioms_list)

['a tight corner', 'pave the way', 'know the ropes', "break sb's heart", 'the jury is (still) out', 'slip your mind', 'have a chip on your shoulder', 'pull strings', 'put your feet up', 'a walk down memory lane', 'know your stuff', 'beyond your wildest dreams', 'with bated breath', 'pull your weight', 'blow your mind', 'know sth inside out', 'touch and go', 'play games', 'easier said than done', 'behind closed doors', 'a bumpy ride', 'at face value', "catch sb's eye", 'run riot', 'with flying colours', 'set the record straight', 'throw caution to the wind', 'bring a lump to your throat', 'keep body and soul together', 'go hand in hand', "cross sb's path", 'leave someone to their own devices', 'get your act together', 'cross your mind', 'call it a day', 'a bone of contention']


In [20]:
import os
import json
import time
from random import randint

# 创建文件夹found_idioms，如果不存在
if not os.path.exists('found_idioms'):
    os.makedirs('found_idioms')

# 遍历found_idioms_list
for idiom in found_idioms_list:
    time.sleep(randint(1, 3))
    definition = get_definition(idiom, 'idiom')
    file_path = os.path.join('found_idioms', f"{idiom}.json")
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(definition, f, ensure_ascii=False, indent=4)

In [22]:
found_pv_list = []
with open('found_pv.json', 'r', encoding='utf-8') as file:
    found_pv_list = json.load(file)
print(found_pv_list)

['belong to sth', 'go back', 'make sb out', 'seal sth off', 'count on sb', 'stay over', 'complain of sth', 'keep sth to yourself', 'close in', 'stand up to sth', 'back sth up', 'work on sth', 'back off', 'mess sth up', 'drift off', 'go on', 'tell sb off', 'get carried away', 'keep sth back', 'cut sth down', 'cheer (sb) up', 'devote sth to sth', 'call for sth', 'stumble upon sb', 'hold down a job', 'let off sb', 'put sth down', 'join in (sth)', 'write down sth', 'figure sb out', 'give (sth) up', 'slip out', 'have heard of sth', 'kick off', 'follow sth up', 'agree with sth', 'end up', 'splash out', 'lock sb in', 'take sth out', 'keep to sth', 'go by', 'split up', 'call sth off', 'fix sth up', 'associate with sb', 'show up', 'get sth across', 'run through sth', 'get round to sth', 'keep sth off sth)', 'squeeze in sth', 'clean sb) up', 'talk sth over', 'press on', 'subject sth to sth', 'take off', 'blow (sth) out', 'wear out sb', 'back (sth) up', 'open up', 'go over sth', 'resign yourself 

In [25]:
found_words_list = []
with open('found_words.json', 'r', encoding='utf-8') as file:
    found_words_list = json.load(file)
print(found_words_list)

['deduction', 'uninteresting', 'behaviour', 'attend', 'she', 'delight', 'impulse', 'saucepan', 'humiliated', 'counterpart', 'funeral', 'merge', 'bug', 'skirt', 'arise', 'railway', 'succeed', 'password', 'verify', 'whole', 'phenomenon', 'bomb', 'enemy', 'either', 'simplification', 'donkey', 'cope', 'editor', 'objective', 'sideways', 'nostalgia', 'member', 'vocabulary', 'handsome', 'blow', 'spotlight', 'onion', 'desire', 'tent', 'relate', 'interval', 'workshop', 'unsuccessfully', 'techno', 'instruct', 'celebration', 'rainy', 'pilot', 'unanimous', 'rainforest', 'scientist', 'list', 'prestige', 'combat', 'underwater', 'divorce', 'second-hand', 'justifiable', 'transaction', 'obtain', 'hope', 'scheduled', 'undeniable', 'tool', 'shed', 'prostitute', 'grin', 'severe', 'grown-up', 'tabloid', 'largely', 'complete', 'humility', 'area', 'rightly', 'thunder', 'obesity', 'notice', 'identity card', 'dense', 'mean', 'commercial', 'industry', 'continuity', 'conductor', 'shampoo', 'atomic', 'discriminat

In [30]:
found_phrases_list = []
with open('found_phrases.json', 'r', encoding='utf-8') as file:
    found_phrases_list = json.load(file)
print(found_phrases_list)

['in case of sth', 'speak of sth', 'pretty well', 'not say a word', 'not be up to much', 'be meant to do sth', 'keep sth in mind', 'at your convenience', 'take exception to sth', 'apart from', 'regards', 'not least', 'to all intents and purposes', 'Yours sincerely', 'in your heart', 'No problem.', 'another', 'round the corner', 'make a go of sth', 'limitations', 'the bulk of sth', 'fair play', 'of no consequence', 'take sb seriously', 'be no mean feat', 'the sixties', 'on the dot', 'the perfect (tense)', 'Dear Sir', 'at all costs', 'in the end', 'be sure of sth', 'all over somewhere', 'rise to your feet', 'on fire', 'behind bars', "sb's looks", 'in itself', 'all is well', 'thick and fast', 'get off', 'as though', 'body and soul', 'in other words', 'needless to say', 'the the Tube', 'excel yourself', 'keep sb company', 'break the news to sb', 'break a record', 'above all', 'be in good taste', 'not be on speaking terms', 'each other', 'lose track', 'give birth', 'within limits', 'the eig

In [24]:
import time
from random import randint

if not os.path.exists('found_pvs'):
    os.makedirs('found_pvs')

for pv in found_pv_list:
    time.sleep(randint(1, 3))
    definition = get_definition(pv, 'phrasal-verb')
    file_path = os.path.join('found_pvs', f"{pv}.json")
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(definition, f, ensure_ascii=False, indent=4)

In [31]:
import os
import json
import time
from random import randint

# 创建文件夹found_phrases，如果不存在
if not os.path.exists('found_phrases'):
    os.makedirs('found_phrases')

# 遍历found_phrases_list
for phrase in found_phrases_list:
    time.sleep(randint(1, 3))
    definition = get_definition(phrase, 'phrase')
    file_path = os.path.join('found_phrases', f"{phrase}.json")
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(definition, f, ensure_ascii=False, indent=4)

In [36]:
import os
import json
import time
from random import randint

# 创建文件夹found_words，如果不存在
if not os.path.exists('found_words'):
    os.makedirs('found_words')

# 遍历found_words_list
for word in found_words_list:
    time.sleep(randint(1, 3))
    definition = get_definition(word, 'word')
    file_path = os.path.join('found_words', f"{word}.json")
    with open(file_path, 'w', encoding='utf-8') as f:
        try:
            json.dump(definition, f, ensure_ascii=False, indent=4)
        except Exception as e:
            print(f"Error processing word: {word}")
            print(e)
        # json.dump(definition, f, ensure_ascii=False, indent=4)

Error processing word: unlock
Object of type set is not JSON serializable


In [37]:
import os

# 统计found_words目录下的文件数量
found_words_dir = 'found_words'
file_count = len([name for name in os.listdir(found_words_dir) if os.path.isfile(os.path.join(found_words_dir, name))])

print(f"Number of files in '{found_words_dir}' directory: {file_count}")

Number of files in 'found_words' directory: 667
